TV나 영화를 모두 내려받지 않고도 시청할 수 있게 네트워크를 통해 스트리밍 하는 미디어 서버 제작

플레이 중인 비디오를 플레이 시간상 앞이나 뒤로 이동해서 일부를 건너뛰거나 반복하는 기능이 있다

클라이언트 프로그램에서 사용자가 선택한 시간에 대응하는 덩어리를 서버에 요청해 이 기능을 구현할 수 있다

In [ ]:
def timecode_to_index(video_id, timecode):
    ...
    # 비디오 데이터의 바이트 오프셋을 반환한다
    

def request_chunk(video_id, byte_offset, size):
    ...
    # video_id 에 대한 비디오 데이터 중에서 바이트 오프셋부터 size만큼을 반환한다
    
video_id = ...
timecode = '01:09:14:28'
byte_offset = timecode_to_index(video_id, timecode)
size = 20 * 1024 * 1024 
video_data = request_chunk(video_id, byte_offset, size)
# request_chunk 요청을 받아서 요청에 해당하는 20MB에 해당하는 데이터를 돌려주는 서버측 핸들러?

In [ ]:
socket = ...
video_data = ...
byte_offset = ...
size = 20 * 1024 * 1024

chunk = video_data[byte_offset:byte_offset + size]
socket.send(chunk)

# 이 코드의 지연 시간과 스루풋은 video_data에서 20MB의 비디오 덩어리를 가져오는데 걸리는 시간과

# 이 데이터를 클라이언트에게 송신하는 데 걸리는 시간이라는 두 가지 요인에 의해 결정 된다.

# 최대 성능을 알아보려면 소켓 송신 부분은 무시하고 데이터 덩어리를 만들기 위해 bytes 인스턴스를 슬라이싱 하는 방법에 걸리는 시간을 측정하면 된다.



In [1]:
import timeit

def run_test():
    chunk = video_data[byte_offset:byte_offset + size]
    
result = timeit.timeit(
            stmt='run_test()',
            globals=globals(),
            number=100
         ) / 100

print(f'{result:0.9f}초')

NameError: name 'video_data' is not defined

클라이언트에게 보낼 20MB의 슬라이스는 대략 5밀리초가 걸린다

서버의 전체 스루풋이 20 MB / 5밀리초 = 7.3GB/초 라는 뜻이다.

이 서버 에서 병렬로 새 데이터 덩어리를 요청할 수 있는 클라이언트 개수는

1 CPU - 초 / 5밀리초 = 200이다.

이 개수는 asyncio내장 모듈같은 도구가 지원할 수 이는 수만 건의 동시 접속에 비하면 아주 작다.

문제는 기반 데이터를 bytes 인스턴스로 슬라이싱 하려면 메모리를 복사해야하는데, 이 과정이 CPU 시간을 점유한다는 점이다.



memoryview 내장 타입은 CPython의 고성능 버퍼 프로토콜을 프로그램에 노출시켜준다

bytes같은 객체를 통하지않고 하부 데이터 버퍼에 접근할 수 있게 해주는 저수준 C API 이다

memoryview 인스턴스에 가장 좋은 점은 스라이싱하면 데이터를 복사하지 않고 새로운 memoryview 인스턴스를 만들어 준다는 것이다



In [2]:
data = '동해물과 abc 백수산이 마르고 닳도록'.encode("utf-8")
view = memoryview(data)
chunk = view[12:19]

print(chunk)
print('크기:', chunk.nbytes)
print('뷰의 데이터:', chunk.tobytes())
print('내부 데이터:', chunk.obj)

크기: 7
뷰의 데이터: b' abc \xeb\xb0'
내부 데이터: b'\xeb\x8f\x99\xed\x95\xb4\xeb\xac\xbc\xea\xb3\xbc abc \xeb\xb0\xb1\xec\x88\x98\xec\x82\xb0\xec\x9d\xb4 \xeb\xa7\x88\xeb\xa5\xb4\xea\xb3\xa0 \xeb\x8b\xb3\xeb\x8f\x84\xeb\xa1\x9d'


복사가 없는 Zero-copy 연산을 활성화 함으로써 memoryview는 NumPy같은 수치 계산 C확장이나

이 예제 프로그램 같은 I/O 위주 프로그램이 커다란 메모리를 빠르게 처리해야하는 경우 성능을 엄청나게 확장 시킬 수 있다.



In [3]:
video_view = memoryview(video_data)

def run_test():
    chunk = video_view[byte_offset:byte_offset + size]
    # socket.send(chunk)를 호출해야 하지만 벤치마크를 위해 무시한다

result = timeit.timeit(
    stmt='run_test()',
    globals=globals(),
    number=100) / 100

print(f'{result:0.9f} 초')

#
socket = ...       # 클라이언트가 연결한 소켓
video_cache = ...  # 서버로 들어오는 비디오 스트림의 캐시
byte_offset = ...  # 데이터 버퍼 위치
size = 1024 * 1024 # 데이터 덩어리 크기

NameError: name 'video_data' is not defined

결과는 250 나노초이다. 20 MB / 250 나노초 = 164 TB / 초다.

병렬 클라이언트의 경우 이론적으로 1 CPU - 초 / 250 나노초 = 400만 개 까지 지원할 수 있다.

데이터가 반대 방향으로 흐를 경우, 

일부 클라이언트가 여러 사용자에게 방송을 하기 위해 서버로 라이브 비디오 스트림을 보내야한다.

사용자가 가장 최근에 보낸 비디오 데이터를 캐시에 넣고 다른 클레이언트가 캐시에 있는 비디오 데이터를 읽게 해야한다


In [7]:
#
# 아이템 74
#

def timecode_to_index(video_id, timecode):
    return 1243
    # 비디오 데이터의 바이트 오프셋을 반환한다


def request_chunk(video_id, byte_offset, size):
    pass
    # video_id에 대한 비디오 데이터 중에 바이트 오프셋부터 size만큼을 반환한다


video_id = ...
timecode = '01:09:14:28'
byte_offset = timecode_to_index(video_id, timecode)
size = 20 * 1024 * 1024
video_data = request_chunk(video_id, byte_offset, size)

# 책에는 없지만 실행을 시키기 위해 추가한 코드
# 소켓을 에뮬레이션
import os

class NullSocket:
    def __init__(self):
        self.handle = open(os.devnull, 'wb')

    def send(self, data):
        self.handle.write(data)
# 책에는 없지만 실행을 시키기 위해 추가한 코드 끝

socket = ...            # 클라이언트가 연결한 소켓
video_data = ...        # video_id에 해당하는 데이터가 들어 있는 bytes
byte_offset = ...       # 요청받은 시작 위치
size = 20 * 1024 * 1024 # 요정받은 데이터 크기

# 책에는 없지만 실행을 시키기 위해 추가한 코드
socket = NullSocket()
video_data = 100 * os.urandom(1024 * 1024)
byte_offset = 1234
# 책에는 없지만 실행을 시키기 위해 추가한 코드 끝

chunk = video_data[byte_offset:byte_offset + size]
socket.send(chunk)

#
import timeit

video_data = 100 * os.urandom(1024 * 1024)
byte_offset = 1234

def run_test():
    chunk = video_data[byte_offset:byte_offset + size]
    # socket.send(chunk)를 호출해야 하지만 벤치마크를 위해 무시한다

result = timeit.timeit(
    stmt='run_test()',
    globals=globals(),
    number=100) / 100

print(f'{result:0.9f} 초')

#
data = '동해물과 abc 백두산이 마르고 닳도록'.encode("utf8")
view = memoryview(data)
chunk = view[12:19]
print(chunk)
print('크기      :', chunk.nbytes)
print('뷰의 데이터:', chunk.tobytes())
print('내부 데이터:', chunk.obj)

#
video_view = memoryview(video_data)

def run_test():
    chunk = video_view[byte_offset:byte_offset + size]
    # socket.send(chunk)를 호출해야 하지만 벤치마크를 위해 무시한다

result = timeit.timeit(
    stmt='run_test()',
    globals=globals(),
    number=100) / 100

print(f'{result:0.9f} 초')

#
socket = ...       # 클라이언트가 연결한 소켓
video_cache = ...  # 서버로 들어오는 비디오 스트림의 캐시
byte_offset = ...  # 데이터 버퍼 위치
size = 1024 * 1024 # 데이터 덩어리 크기

# 책에는 없지만 실행을 시키기 위해 추가한 코드
# 소켓을 에뮬레이션
class FakeSocket:

    def recv(self, size):
        return video_view[byte_offset:byte_offset+size]

    def recv_into(self, buffer):
        source_data = video_view[byte_offset:byte_offset+size]
        buffer[:] = source_data

socket = FakeSocket()
video_cache = video_data[:]
byte_offset = 1234
# 책에는 없지만 실행을 시키기 위해 추가한 코드 끝

chunk = socket.recv(size)
video_view = memoryview(video_cache)
before = video_view[:byte_offset]
after = video_view[byte_offset + size:]
new_cache = b''.join([before, chunk, after])

#
class FakeSocket:

    def recv(self, size):
        return video_view[byte_offset:byte_offset+size]

    def recv_into(self, buffer):
        source_data = video_view[byte_offset:byte_offset+size]
        buffer[:] = source_data

size = 1024 * 1024
socket = FakeSocket()
video_cache = video_data[:]
byte_offset = 1234

chunk = socket.recv(size)
video_view = memoryview(video_cache)
before = video_view[:byte_offset]
after = video_view[byte_offset + size:]
new_cache = b''.join([before, chunk, after])

def run_test():
    chunk = socket.recv(size)
    before = video_view[:byte_offset]
    after = video_view[byte_offset + size:]
    new_cache = b''.join([before, chunk, after])

result = timeit.timeit(
    stmt='run_test()',
    globals=globals(),
    number=100) / 100

print(f'{result:0.9f} 초')

#
my_bytes = b'hello'
# 오류가 나는 부분. 오류를 보고 싶으면 커멘트를 해제할것
#my_bytes[0] = b'\x79'

#
my_array = bytearray('hello 안녕'.encode("utf8"))
my_array[0] = 0x79
print(my_array)

#
my_array = bytearray('row, row, row your 보트'.encode("utf8"))
my_view = memoryview(my_array)
write_view = my_view[3:13]
write_view[:] = b'-10 bytes-'
print(my_array)

#

video_array = bytearray(video_cache)
write_view = memoryview(video_array)
chunk = write_view[byte_offset:byte_offset + size]
socket.recv_into(chunk)

def run_test():
    chunk = write_view[byte_offset:byte_offset + size]
    socket.recv_into(chunk)

result = timeit.timeit(
    stmt='run_test()',
    globals=globals(),
    number=100) / 100

print(f'{result:0.9f} seconds')

0.004420547 초
크기      : 7
뷰의 데이터: b' abc \xeb\xb0'
내부 데이터: b'\xeb\x8f\x99\xed\x95\xb4\xeb\xac\xbc\xea\xb3\xbc abc \xeb\xb0\xb1\xeb\x91\x90\xec\x82\xb0\xec\x9d\xb4 \xeb\xa7\x88\xeb\xa5\xb4\xea\xb3\xa0 \xeb\x8b\xb3\xeb\x8f\x84\xeb\xa1\x9d'
0.000000802 초
0.103038971 초
bytearray(b'yello \xec\x95\x88\xeb\x85\x95')
bytearray(b'row-10 bytes- your \xeb\xb3\xb4\xed\x8a\xb8')
0.000203719 seconds
